<a href="https://colab.research.google.com/github/newfull5/AI_Poet-KoGPT2/blob/master/2.%20Transfer_Learning!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!git clone https://github.com/SKT-AI/KoGPT2.git

In [0]:
!cd KoGPT2 && pip install -r requirements.txt
!cd KoGPT2 && pip install .

In [0]:
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

In [13]:
if torch.cuda.device_count():
  print('구글 만세!')
  PU = 'cuda'
else:
  PU = 'cpu'

구글 만세!


In [0]:
#토큰화와 인덱싱을해서 리턴하는 클래스
class Data_Set(Dataset):

  def __init__ (self, file_path,vocab,tokenizer):
    self.data = []
    self.vocab = vocab
    self.tokenizer = tokenizer

    f = open(file_path,'r',encoding='utf-8')

    file = f.read()
    file = file.split('\n')

    dataset = []
    now = ''

    for i, line in enumerate(file):
      if i % 30 == 0 and i != 0:
        dataset.append(now)
        now = ''

      now = now + '\n' + line

    for line in dataset:
      tokenized_line = tokenizer(line[:-1])

      indexing_word = [vocab[vocab.bos_token], ]+ vocab[tokenized_line] + [vocab[vocab.eos_token]]
      self.data.append(indexing_word)

    f.close()

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.data[index]

In [20]:
model, vocab = get_pytorch_kogpt2_model()

model.to(torch.device(PU)) #모델 연산 유닛 설정
model.train() #모델 학습모드로 변경

save_path = 'drive/My Drive/Colab Notebooks/KoGPT2_checkpoint/'

using cached model
using cached model


In [0]:
#모델 불러오기 코드 입니다
from transformers import GPT2Config, GPT2LMHeadModel

save_path = 'drive/My Drive/Colab Notebooks/KoGPT2_checkpoint/'

kogpt2_config = {
		"initializer_range": 0.02,
		"layer_norm_epsilon": 0.000001,
		"n_ctx": 1024,
		"n_embd": 768,
		"n_head": 12,
		"n_layer": 12,
		"n_positions": 1024,
		"vocab_size": 50000,
		"activation_function": "gelu"
}

checkpoint = torch.load(save_path+'KoGPT2_checkpoint.tar', map_location=PU)

kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))

kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.train()

kogpt2model.to(torch.device(PU))

model = kogpt2model

In [0]:
#torch.save(model.state_dict,'model_state_dict.pth') #모델의 가중치 값을 저장하는 코드입니다.
#model.load_state_dict(torch.load(save_path+'KoGPT2_checkpoint.tar')) #모델의 가중치 값을 불러오는 코드입니다.

#torch.save(model, PATH) #모델 전체를 저장하는 코드입니다.
#model = torch.load(PATH) #모델 전체를 불러오는 코드입니다.

In [21]:
file_path = 'drive/My Drive/poem_data/dataset_ver3.txt'
tokenizer = SentencepieceTokenizer(get_tokenizer(), num_best=0, alpha=0)

data = Data_Set(file_path, vocab, tokenizer)

dataset = DataLoader(data, batch_size=2, shuffle=True, pin_memory=True)

learning_rate = 0.00005
epochs = 300
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

using cached model


In [22]:
for epoch in range(checkpoint['epoch'], epochs+1):
  cnt = 0

  for data in dataset:
    optimizer.zero_grad()

    data = torch.stack(data)
    data = data.transpose(1,0)
    data = data.to(PU)

    output = model(data,labels=data)
    loss, logits = output[:2]
    loss.backward()
    optimizer.step()

    if cnt % 20 == 0:
      print("[+] epoch : {}, cnt : {}, loss : {} [+]".format(epoch, cnt+1, str(loss)[7:12]))

    if epoch % 20 == 0 and cnt == 1:
      torch.save({
          'epoch': epoch,
          'cnt': cnt,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, save_path+'KoGPT2_checkpoint.tar')
      
    cnt += 1

[+] epoch : 0, cnt : 1, loss : 4.906 [+]
[+] epoch : 0, cnt : 21, loss : 6.181 [+]
[+] epoch : 0, cnt : 41, loss : 5.126 [+]
[+] epoch : 0, cnt : 61, loss : 5.664 [+]
[+] epoch : 0, cnt : 81, loss : 5.171 [+]
[+] epoch : 0, cnt : 101, loss : 5.409 [+]
[+] epoch : 0, cnt : 121, loss : 5.456 [+]
[+] epoch : 0, cnt : 141, loss : 4.590 [+]
[+] epoch : 0, cnt : 161, loss : 5.703 [+]
[+] epoch : 0, cnt : 181, loss : 5.000 [+]
[+] epoch : 0, cnt : 201, loss : 5.415 [+]
[+] epoch : 0, cnt : 221, loss : 5.568 [+]
[+] epoch : 0, cnt : 241, loss : 5.587 [+]
[+] epoch : 0, cnt : 261, loss : 5.855 [+]
[+] epoch : 0, cnt : 281, loss : 5.849 [+]
[+] epoch : 1, cnt : 1, loss : 5.659 [+]
[+] epoch : 1, cnt : 21, loss : 5.136 [+]
[+] epoch : 1, cnt : 41, loss : 5.207 [+]
[+] epoch : 1, cnt : 61, loss : 4.893 [+]
[+] epoch : 1, cnt : 81, loss : 5.839 [+]
[+] epoch : 1, cnt : 101, loss : 5.412 [+]
[+] epoch : 1, cnt : 121, loss : 5.110 [+]
[+] epoch : 1, cnt : 141, loss : 6.012 [+]
[+] epoch : 1, cnt : 16